---
title: "XGboost classification model"
subtitle: "with whiskynote.be data"
author: "Tony Duan"

execute:
  warning: false
  error: false
  eval: false

format:
  html:
    toc: true
    toc-location: right
    code-fold: show
    code-tools: true
    number-sections: true
    code-block-bg: true
    code-block-border-left: "#31BAE9"
---


In [ ]:
import tensorflow as tf

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

from siuba.siu import call
from siuba import _, mutate, filter, group_by, summarize,show_query
from siuba import *

from siuba.data import mtcars,penguins

# read in data

In [ ]:
import pandas as pd
data=pd.read_excel('./output/all_page_bottle_list_all.xlsx')

In [ ]:
list(data)

In [ ]:
data.info()

In [ ]:
import re
data001=data>> filter(_.all_page_score >=70
                      ,_.all_page_score <100
                      ,_.bottle_review_Nose !='no comment'
                      ,_.bottle_review_Mouth !='no comment'
                      ,_.bottle_review_Finish !='no comment'
                      ) >>mutate(
                        review_flag=if_else(_.all_page_score>=90,1,0)
                      ,review=_.bottle_review_Nose+_.bottle_review_Mouth+_.bottle_review_Finish
                      )>>mutate(review=_.bottle_name+_.review.str.lower().str.replace('nose:','').str.replace('mouth:','').str.replace('finish:','').str.replace('.','').str.replace(',','').str.replace('(','').str.replace(')','').str.replace('-','').str.replace('apples','apple').str.replace('oranges','orange').str.replace('sweetness','sweet').str.replace('fruits','fruit'))>>mutate(review_len=_.review.str.count(' ') + 1)


In [ ]:
data001 = data001.dropna(subset=["review_flag", "review"])

In [ ]:
data001.review_flag = data001.review_flag.astype(int)

In [ ]:
data001.info()

# shuffle data

In [ ]:
data002=data001.sample(frac=1)

In [ ]:
data002.to_excel('data002.xlsx')

In [ ]:
data002.info()

In [ ]:
review=data002['review'].tolist()

In [ ]:
review[2]

In [ ]:
review_flag=data002["review_flag"].tolist()

In [ ]:
review_score=data002["all_page_score"].tolist()

In [ ]:
review_flag[2]

In [ ]:
from collections import Counter
Counter(review_flag)

In [ ]:
print(len(review))

print(len(review_flag))

# transfer data

In [ ]:
import tensorflow as tf
import numpy as np 
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Initialize the Tokenizer class
tokenizer = Tokenizer()

# Generate the word index dictionary
tokenizer.fit_on_texts(review)

# Define the total words. You add 1 for the index `0` which is just the padding token.
total_words = len(tokenizer.word_index) + 1

In [ ]:
print(f'total words: {total_words}')

In [ ]:
# Convert labels lists to numpy array
review_flag_final = np.array(review_flag)
review_score_final = np.array(review_score)

In [ ]:
# Parameters
vocab_size = 7000
max_length = 300
embedding_dim = 16
#trunc_type='pre'
trunc_type='post'
oov_tok = "<OOV>"

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the Tokenizer class
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

# Generate the word index dictionary for the training sentences
tokenizer.fit_on_texts(review)
word_index = tokenizer.word_index

# Generate and pad the training sequences
sequences = tokenizer.texts_to_sequences(review)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

In [ ]:
len(review[4])

In [ ]:
len(padded[4])

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])            

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [ ]:
print(decode_review(padded[0]))

after tokenizer

In [ ]:
print(sequences[0])

In [ ]:
review_flag[0]

# using 4000 to train and 633 to test

In [ ]:
padded_train=padded[0:4000]
padded_test=padded[4000:]

In [ ]:
review_flag_final_train=review_flag_final[0:4000]
review_flag_final_test=review_flag_final[4000:]

In [ ]:
review_socre_final_train=review_score_final[0:4000]
review_socre_final_test=review_score_final[4000:]

## total

In [ ]:
len(padded)
len(review_flag_final)

## train

In [ ]:
len(padded_train)
len(review_flag_final_train)

## test

In [ ]:
len(padded_test)
len(review_flag_final_test)

In [ ]:
sum(review_flag_final_test)

## if all guess lower than 90 points then 0.75 accuracy 

In [ ]:
(len(review_flag_final_test)-sum(review_flag_final_test))/len(review_flag_final_test)

# XB bootst model


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
import time
from siuba import *

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [ ]:
from xgboost import XGBClassifier
ml_model = XGBClassifier()
ml_model

# define pipline

In [ ]:
#ml_model.fit(padded_train,review_flag_final_train)

In [ ]:
pipeline = Pipeline(
  steps=[
      
         ('model', ml_model)
         ]
)

pipeline

In [ ]:
from sklearn.model_selection import GridSearchCV


parameters = {
        'model__learning_rate': [0.08,0.1],
        'model__max_depth': [9,10,20],
        'model__min_child_weight': [3,5,8],
        'model__subsample': [0.7,0.9],
        
       # 'model__colsample__bytree': [0.5, 0.7],
       
        'model__n_estimators' : [100,200],
        'model__objective': ['reg:squarederror']
    }

In [ ]:
import itertools
a = parameters.values()
combinations = list(itertools.product(*a))
len(combinations)

In [ ]:
GridCV = GridSearchCV(pipeline
                ,parameters
                ,scoring='accuracy'
                , cv=3, n_jobs=-1)

# train model

In [ ]:
start_time = time.time()

GridCV.fit(padded_train, review_flag_final_train)

end_time = time.time()
duration = end_time - start_time
duration

In [ ]:
model_ml = GridCV.best_estimator_

In [ ]:
#Using predict method to test the model
Y_pred_dt = model_ml.predict(padded_test) #always gets x and retuns y
Y_pred_dt

In [ ]:
# Accuracy = true negatives + true positives / true positives + false positives + true negatives + false negatives
# Here is another way to find the accuracy score
from sklearn import metrics
accuracy = metrics.accuracy_score(review_flag_final_test,Y_pred_dt)  
accuracy

In [ ]:
import seaborn as sns
confusion_matrix_dt = metrics.confusion_matrix(review_flag_final_test,Y_pred_dt)
confusion_matrix_dt

# resource:

https://www.tensorflow.org/tutorials/keras/regression
